In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [2]:
# program = "graph_coloring"
program = "maximal_matching"
# program = "dijkstra_token_ring"
# graph_name = "graph_7"
graph_name = "graph_10"
# graph_name = "implicit_graph_n10"

no_simulations = 10000
fault_interval = 2
sched = 0

selected_nodes = [0, 4, 9]

simulation_type = "controlled_at_node_amit_v2"
limits = 100

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__{simulation_type}__{limits}"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,30,320.248989
1,31,307.700189
2,32,295.615244
3,33,283.977007
4,33,272.768967
...,...,...
95,97,-9.756629
96,98,-10.107090
97,98,-10.444597
98,99,-10.769629


In [5]:
dfs[0]

,Steps,exp_fit
0,30,163.918399
1,31,160.897192
2,32,157.951252
3,33,155.078704
4,33,152.277719
...,...,...
95,97,53.685126
96,98,53.410142
97,98,53.142009
98,99,52.880555


In [6]:
dfs[1]

,Steps,exp_fit
0,30,182.031741
1,31,177.333284
2,32,172.799876
3,33,168.425720
4,33,164.205220
...,...,...
95,97,52.756737
96,98,52.599497
97,98,52.447781
98,99,52.301394


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__4,exp_fit__9
Steps,,,
30,163.918399,182.031741,320.248989
31,160.897192,177.333284,307.700189
32,157.951252,172.799876,295.615244
33,155.078704,168.425720,283.977007
33,155.078704,168.425720,272.768967
...,...,...,...
98,53.142009,52.599497,-10.444597
98,53.142009,52.447781,-10.107090
98,53.142009,52.447781,-10.444597


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 4,Exponential fit of node 9
Steps,,,
30,163.918399,182.031741,320.248989
31,160.897192,177.333284,307.700189
32,157.951252,172.799876,295.615244
33,155.078704,168.425720,283.977007
33,155.078704,168.425720,272.768967
...,...,...,...
98,53.142009,52.599497,-10.444597
98,53.142009,52.447781,-10.107090
98,53.142009,52.447781,-10.444597


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [11]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}__{fault_interval}__{limits}.png",
    bbox_inches="tight",
)
plt.close()